## Global Settings and Imports

In [8]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

In [1]:
import argparse
import yaml
from dotmap import DotMap
from os import path
import numpy as np
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer
from models.lstur import LSTUR
from models.nrms import NRMS
from models.naml import NAML
from models.naml_simple import NAML_Simple
from models.sentirec import SENTIREC
from models.robust_sentirec import ROBUST_SENTIREC
from data.dataset import BaseDataset
from tqdm import tqdm

## Prepare parameters

In [2]:
args = argparse.Namespace(
    config = "config/model/sentirec/vader_manual1.yaml",
    resume = None
)

with open(args.config, 'r') as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)
    config = DotMap(config)

assert(config.name in ["lstur", "nrms", "naml", "naml_simple", "sentirec", "robust_sentirec"])

pl.seed_everything(1234)

logger = TensorBoardLogger(
    **config.logger
)
checkpoint_callback = ModelCheckpoint(
    **config.checkpoint
)

Seed set to 1234


## Load data

In [3]:
preprocess_path = f"{config.preprocess_data_path}/{config.dataset_size}/"

train_dataset = BaseDataset(
    path.join(preprocess_path+config.train_behavior),
    path.join(preprocess_path+config.train_news), 
    config)
val_dataset = BaseDataset(
    path.join(preprocess_path+config.val_behavior),
    path.join(preprocess_path+config.train_news), 
    config) 
train_loader = DataLoader(
    train_dataset,
    **config.train_dataloader)
val_loader = DataLoader(
    val_dataset,
    **config.val_dataloader)

100%|██████████| 1/1 [00:00<00:00, 499.14it/s]


In [4]:
# load embedding pre-trained embedding weights
embedding_weights=[]
with open(path.join(preprocess_path+config.embedding_weights), 'r') as file: 
    lines = file.readlines()
    for line in tqdm(lines):
        weights = [float(w) for w in line.split(" ")]
        embedding_weights.append(weights)
pretrained_word_embedding = torch.from_numpy(
    np.array(embedding_weights, dtype=np.float32)
)

100%|██████████| 3246/3246 [00:00<00:00, 20400.24it/s]


## Create model

In [5]:
print(config.name)
if config.name == "lstur":
    model = LSTUR(config, pretrained_word_embedding)
elif config.name == "nrms":
    model = NRMS(config, pretrained_word_embedding)
elif config.name == "naml":
    model = NAML(config, pretrained_word_embedding)
elif config.name == "naml_simple":
    model = NAML_Simple(config, pretrained_word_embedding)
elif config.name == "sentirec":
    model = SENTIREC(config, pretrained_word_embedding)
elif config.name == "robust_sentirec":
    model = ROBUST_SENTIREC(config, pretrained_word_embedding)

sentirec


## Train model

In [6]:
early_stop_callback = EarlyStopping(
    **config.early_stop
)
if args.resume is not None:
    model = model.load_from_checkpoint(
        args.resume, 
        config=config, 
        pretrained_word_embedding=pretrained_word_embedding)
    trainer = Trainer(
        **config.trainer,
        callbacks=[early_stop_callback, checkpoint_callback],
        logger=logger,
        resume_from_checkpoint=args.resume
    )
else:
    trainer = Trainer(
        **config.trainer,
        callbacks=[early_stop_callback, checkpoint_callback],
        logger=logger,
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(
    model=model, 
    train_dataloaders=train_loader, 
    val_dataloaders=val_loader
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                                   | Type             | Params | Mode 
-------------------------------------------------------------------------------------
0  | news_encoder                           | TimeDistributed  | 1.4 M  | train
1  | user_encoder                           | UserEncoder      | 421 K  | train
2  | sentiment_predictor                    | Linear           | 301    | train
3  | val_performance_metrics                | MetricCollection | 0      | train
4  | val_sentiment_diversity_metrics_vader  | MetricCollection | 0      | train
5  | val_sentiment_diversity_metrics_bert   | MetricCollection | 0      | train
6  | test_performance_metrics               | MetricCollection | 0      | train
7  | test_sentiment_diversity_metrics_vader | MetricCollection | 0      | train
8  | test_sentiment_diversity_metrics_bert  | MetricCollection | 0      | train
9  | test_topic_diversity_metrics           | MetricCollection | 0     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\USER\anaconda3\envs\newsrec\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\USER\anaconda3\envs\newsrec\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\USER\anaconda3\envs\newsrec\lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 15, global step 16: 'val_auc_epoch' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 16, global step 17: 'val_auc_epoch' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_auc_epoch did not improve in the last 5 records. Best score: 0.867. Signaling Trainer to stop.
Epoch 17, global step 18: 'val_auc_epoch' was not in top 3


In [8]:
# 한 impression 데이터의 history에 포함된
# 모든 뉴스의 제목이 저장된 배치 데이터의 길이.
# history가 설정값보다 더 적을 경우,
# 빈 공간은 padding값(0)으로 채워집니다.
len(train_dataset[0]['h_title'])

200

In [9]:
# 모든 뉴스의 token 길이 또한 고정됩니다.
# 마찬가지로 모든 빈칸은 padding값(0)으로 채워집니다. 
len(train_dataset[0]['h_title'][0])

100